In [1]:
import sys
sys.path.insert(0,'..')

In [43]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import torch
import re

from src.model_functions import model_factory
from src.utils import get_param_from_name

from IPython.display import clear_output

In [5]:
output_folder = "charts"
model_type = "bertbase" # bertl4

n_seeds = 5
par_idx = 0

folder = f"/share/home/lukash/pan16/{model_type}/cp_modular"
experiment_name = "modular-diff_pruning_0.01-bert-base-uncased-64-2e-05-sp_pen1.25e-07-weighted_loss_prot-gender_age-seed{}.pt"

In [8]:
diff_weights = []
base_weights = []
sparsity = []
for seed in range(n_seeds):
    filepath = os.path.join(folder, experiment_name.format(seed))
    m = model_factory(filepath)
    diff_weights.append(m.get_diff_weights(idx=par_idx, as_module=False))
    base_weights.append(m.get_base_weights(as_module=False))
    sparsity.append(m._count_non_zero_params(idx=par_idx))
clear_output()

In [11]:
sparsity = torch.tensor(sparsity)
sparsity[:,2] / sparsity[:,0]

tensor([8.2580e-05, 8.5977e-05, 8.3109e-05, 8.8800e-05, 1.0740e-04])

In [49]:
filepath = os.path.join(folder, experiment_name.format(0))
m = model_factory(filepath, debiased=True, debiased_par_idx=0) #, remove_parametrizations=True
clear_output()

In [31]:
bw = dict(base_weights[0])
dw = dict(diff_weights[0])
nonzero = {k:v for k,v in dw.items() if v.sum()>0}

In [32]:
nonzero.keys()

dict_keys(['encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.attention.output.dense.weight', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.2.attention.self.query.weight', 'encoder.layer.2.attention.output.dense.weight', 'encoder.layer.3.attention.self.key.weight', 'encoder.layer.3.attention.output.LayerNorm.weight', 'encoder.layer.3.output.dense.weight', 'encoder.layer.4.intermediate.dense.weight', 'encoder.layer.4.output.dense.weight', 'encoder.layer.5.attention.output.dense.weight', 'encoder.layer.5.intermediate.dense.weight', 'encoder.layer.6.attention.self.key.weight', 'encoder.layer.7.attention.self.value.weight', 'encoder.layer.7.attention.output.dense.weight', 'encoder.layer.7.output.dense.weight', 'encoder.layer.8.attention.self.value.weight', 'encoder.layer.8.output.LayerNorm.bias', 'encoder.layer.9.attention.self.key.weight', 'encoder.layer.10.attention.self.query.weight', 'encoder.layer.10.attention.self.valu

In [55]:
k = 'encoder.layer.7.attention.self.value.weight'

In [62]:
idx_nonzero = nonzero[k].nonzero()
idx_nonzero

tensor([[  1, 381],
        [  9, 544],
        [ 16, 308],
        [ 23, 381],
        [ 25, 381],
        [ 28, 308],
        [ 31, 381],
        [ 42, 381],
        [ 49, 308],
        [ 51, 381],
        [ 60, 445],
        [ 65, 381],
        [ 66, 381],
        [ 74, 308],
        [ 76, 738],
        [ 87, 582],
        [ 91, 381],
        [107, 308],
        [118, 757],
        [122, 381],
        [125, 381],
        [135, 381],
        [136, 291],
        [136, 381],
        [139, 381],
        [140, 381],
        [142, 308],
        [142, 381],
        [146, 539],
        [147, 381],
        [150, 381],
        [153, 381],
        [156, 381],
        [158, 381],
        [162,  20],
        [162, 308],
        [172, 381],
        [173, 188],
        [181, 381],
        [188, 477],
        [189, 308],
        [190, 381],
        [211, 381],
        [213, 381],
        [221, 381],
        [225, 381],
        [228, 381],
        [233, 381],
        [236, 308],
        [237, 381],


In [63]:
dw[k][1,381]

tensor(0.0058)

In [64]:
bw[k][1,381]

tensor(0.0026)

In [65]:
p = get_param_from_name(m.encoder, k).data

In [66]:
p[1,381]

tensor(0.0084)